In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import torch

Create raw data

In [2]:
from metal.utils import hard_to_soft

N = 1200
X = np.random.random((N,2)) * 2 - 1
Y = np.zeros((N,2))
Y[:,0] = (X[:,0] > X[:,1] + 0.5).astype(int) + 1
Y[:,1] = (X[:,0] > X[:,1] + 0.25).astype(int) + 1

X = torch.tensor(X, dtype=torch.float)
Y = torch.tensor(Y, dtype=torch.short)

Separate into splits

In [3]:
X_train = X[:1000]
X_dev   = X[1000:1100]
X_test  = X[1100:]

Y_train = [Y[:1000, 0],     Y[:1000, 1]]
Y_dev   = [Y[1000:1100, 0], Y[1000:1100, 1]]
Y_test  = [Y[1100:, 0],     Y[1100:, 1]]

In [16]:
from metal.utils import hard_to_soft
Y_train = [hard_to_soft(Y_t, k=2) for Y_t in Y_train]

Train and score

In [18]:
from metal.structs import TaskGraph

edges = [(0,1)]
cards = [2,2]
tg = TaskGraph(edges, cards)

In [19]:
from metal.end_model import EndModel

em = EndModel(
    label_map=[[1,2],[1,2]], 
    task_graph=tg,
    seed=1,
    dropout=0.0,
    head_layers=[2,3],
    pass_predictions=True,
)

Reaffirming seed=1
Reaffirming dropout=0.0
Reaffirming head_layers=[2, 3]
Reaffirming pass_predictions=True


In [20]:
em.train(X_train, Y_train, X_dev, Y_dev, n_epochs=10)

Reaffirming n_epochs=10
EndModel(
  (layers): ModuleList(
    (layer0): Sequential(
      (0): IdentityModule()
      (1): ReLU()
      (2): BatchNorm1d(2, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (layer1): Sequential(
      (0): Linear(in_features=2, out_features=10, bias=True)
      (1): ReLU()
      (2): BatchNorm1d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (layer2): Sequential(
      (0): Linear(in_features=10, out_features=10, bias=True)
      (1): ReLU()
      (2): BatchNorm1d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (layer3): Sequential(
      (0): Linear(in_features=10, out_features=2, bias=True)
      (1): ReLU()
      (2): BatchNorm1d(2, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
  )
  (heads): ModuleList(
    (0): Linear(in_features=10, out_features=2, bias=True)
    (1): Linear(in_features=4, out_features=2, bias=True)
  )
  (criteria): SoftCr

In [21]:
em.score(X_test, Y_test, reduce=None)
em.score(X_test, Y_test, reduce='mean')

Accuracy (t=0): 0.910
Accuracy (t=1): 0.930


[0.91, 0.93]

Accuracy: 0.920


0.92